<a href="https://colab.research.google.com/github/mhuckvale/pals0039/blob/master/Answers_4_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[![PALS0039 Logo](https://www.phon.ucl.ac.uk/courses/pals0039/images/pals0039logo.png)](https://www.phon.ucl.ac.uk/courses/pals0039/)

#Exercise 4.3 Answers

(a) Implement age prediction

In [0]:
# test DNN for age prediction
#
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%tensorflow_version 2.x
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout
#
# load a CSV file and divide into feats and labels
def LoadCSV(filename,labelname,featcolstart):
	df=pd.read_csv(filename,sep=',')
	gender=pd.Categorical(df['GENDER'])
	feats=np.append(np.array(df.iloc[:,featcolstart:]),np.reshape(gender.codes,(len(df),1)),axis=1)
	labels=df[labelname].to_numpy()
	p = np.random.permutation(feats.shape[0])
	return labels[p],feats[p]
#
trainLabels,trainFeats=LoadCSV("https://www.phon.ucl.ac.uk/courses/pals0039/data/age-train.csv","AGE",4);
print(trainLabels.shape)
print(trainFeats.shape)
print(trainLabels[1:10])
testLabels,testFeats=LoadCSV("https://www.phon.ucl.ac.uk/courses/pals0039/data/age-test.csv","AGE",4);
#
isize=trainFeats.shape[1];
osize=1
print(isize,osize)
#
# normalise
mn=np.mean(trainFeats,axis=0)
sd=np.std(trainFeats,axis=0)
print("sd min",np.min(sd),"max",np.max(sd));
sd[(sd<0.001)]=1
trainFeats=(trainFeats-mn)/sd;
testFeats=(testFeats-mn)/sd;
trainLabels=trainLabels/100
testLabels=testLabels/100
#
model = Sequential()
model.add(Dense(128,activation='tanh',input_shape=(isize,)))
model.add(Dropout(0.5))
model.add(Dense(64,activation='tanh'))
model.add(Dense(osize, activation='linear'))
model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mae'])
print(model.summary())
#
model.fit(trainFeats,trainLabels, epochs=100, batch_size=32, validation_data=(testFeats,testLabels))
#
testPred=model.predict(testFeats)
#
x=testLabels*100
y=testPred*100
colours=['red','blue']
cols=list(colours[int(i)] for i in testFeats[:,isize-1])
plt.scatter(x=x,y=y,c=cols,s=20,alpha=0.5);
poly = np.polyfit(x, y, 1)
x=np.array((np.min(x),np.max(x)))
y=poly[1]+poly[0]*x
plt.plot(x,y,':r')
plt.xlabel("Actual Age (yr)");
plt.ylabel("Predicted Age (yr)");
plt.grid()
plt.title("Age prediction");
plt.show()
